# Triage network workflow

Tested with Python 3.6.7 (from miniconda)

## Part 0. Dependencies

In [ ]:
%%bash

pip install numpy seaborn sklearn-evaluation tensorflow
pip install git+git://github.com/edublancas/dstools
pip install git+git://github.com/paninski-lab/yass@99cff0c08c11b281b63e2fa2f9c27de66f2d6e48

In [ ]:
%%bash

mkdir -p /tmp/spike-sorting
curl https://dl.dropboxusercontent.com/s/sylnygjmcvkmi4z/templates.npy?dl=0 -o /tmp/spike-sorting/templates.npy
curl https://dl.dropboxusercontent.com/s/smk83ob73y9z7p0/config.yaml?dl=0 -o /tmp/spike-sorting/config.yaml
curl https://dl.dropboxusercontent.com/s/mfp5vcu9b53ws91/noise_cov.npz?dl=0 -o /tmp/spike-sorting/noise_cov.npz
curl https://dl.dropboxusercontent.com/s/k9qa7vttuzrsmr4/geometry.txt?dl=0 -o /tmp/spike-sorting/geometry.txt

## Part 1. Train/Test set creation

In [ ]:
import datetime
import logging
from pathlib import Path
from os.path import expanduser
from os import path

import yass
from yass import read_config
from yass.augment import make
from yass.neuralnetwork import NeuralNetTriage
from yass.batch import RecordingsReader
from yass.augment.noise import noise_cov
from yass.templates import TemplatesProcessor
from yass.geometry import make_channel_index


import seaborn as sns
import numpy as np
from dstools import plot
import sklearn_evaluation.plot as skplot

# logging.basicConfig(level=logging.DEBUG)

In [ ]:
path_to_data = expanduser('~/data')
path_to_experiment = path.join(path_to_data, 'retinal/sample_output')
path_to_standarized = path.join(path_to_experiment,
                                'preprocess', 'standarized.bin')

In [ ]:
yass.set_config('/tmp/spike-sorting/config.yaml',
                '/tmp/spike-sorting/output')
CONFIG = read_config()

### 1.1 Loading templates

In [ ]:
raw_templates = np.load('/tmp/spike-sorting/templates.npy')
n_templates, waveform_length, _ = raw_templates.shape
print(raw_templates.shape)

In [ ]:
# crop spatially
processor = TemplatesProcessor(raw_templates)
templates = (processor
            .crop_spatially(CONFIG.neigh_channels, CONFIG.geom)
            .values)
templates.shape

In [ ]:
plot.grid_from_array(templates, axis=0, auto_figsize=4,
                     max_cols=3, elements=9)

### 1.2 Estimating noise covariance structure

In [ ]:
ch_idx = make_channel_index(CONFIG.neigh_channels, CONFIG.geom)
selected_channels = ch_idx[0]
selected_channels

In [ ]:
LOAD_NOISE_COV = True


if LOAD_NOISE_COV:
    cov = np.load('/tmp/spike-sorting/noise_cov.npz')
    spatial_sig, temporal_sig = cov['spatial_sig'], cov['temporal_sig']
else:
    rec = RecordingsReader(path_to_standarized, loader='array').data[:, selected_channels]
    (spatial_sig,
     temporal_sig) = noise_cov(rec, templates.shape[1], templates.shape[1])

### 1.3 Make dataset

In [ ]:
min_amplitude = 4
max_amplitude = 60

n_clean_per_template = 150
n_collided_per_spike = 1
max_shift = 15
min_shift = 4

probabilities = (0.8, 0.05, 0.05, 0.05, 0.05)

In [ ]:
X, y = make.training_data_triage(templates,
                                       min_amplitude,
                                       max_amplitude,
                                       n_clean_per_template,
                                       n_collided_per_spike,
                                       max_shift,
                                       min_shift,
                                       spatial_sig,
                                       temporal_sig,
                                       {'probabilities': probabilities},
                                       {'return_metadata': True})

In [ ]:
X.shape, y.shape

### 1.4 Plot some examples

In [ ]:
# positive examples: single spikes
plot.grid_from_array(X[y == 1], axis=0,
                     elements=9, auto_figsize=3)

In [ ]:
# negative examples: collisions
plot.grid_from_array(X[y == 0], axis=0,
                     elements=9, auto_figsize=3)

## Part 2. Network training

In [ ]:
filters_size = [64, 32]
threshold = 0.5
_, waveform_length, n_neighbors =  X.shape

model = NeuralNetTriage('/tmp/spike-sorting/my-triage-network.ckpt', filters_size,
                        waveform_length, n_neighbors, threshold,
                        n_iter=5000, n_batch=512, l2_reg_scale=0.00000005,
                        train_step_size=0.001)

In [ ]:
model.fit(X, y)

### Part 3. Network evaluation

In [ ]:
preds = model.predict(model.x_test)

In [ ]:
skplot.confusion_matrix(model.y_test, preds, normalize=True, target_names=['Noise', 'Spike'])

### Part 4. Cleanup

In [ ]:
! rm -rf /tmp/spike-sorting